In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GRU, Dense, Bidirectional, Dropout, LayerNormalization, LeakyReLU
import random
from tensorflow.keras.callbacks import EarlyStopping

try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.TPUStrategy(tpu)
    print("Running on TPU")
except ValueError:
    strategy = tf.distribute.get_strategy()
    print("Running on CPU/GPU")


train_df = pd.read_csv("Train Data.csv")
test_df = pd.read_csv("Test Datas.csv")


MAX_VOCAB_SIZE = 30000
tokenizer = Tokenizer(num_words=MAX_VOCAB_SIZE, oov_token="<OOV>")
tokenizer.fit_on_texts(train_df['SENTENCES'])
word_index = tokenizer.word_index


vocab_size = min(len(word_index), MAX_VOCAB_SIZE)


train_sequences = tokenizer.texts_to_sequences(train_df['SENTENCES'])


max_len = 256
train_padded = pad_sequences(train_sequences, maxlen=max_len, padding='post')


X, y = [], []
for seq in train_sequences:

    seq_padded = pad_sequences([seq], maxlen=max_len, padding='post')[0]

    nonzero_indices = [i for i, val in enumerate(seq_padded) if val != 0]
    for idx in nonzero_indices:

        seq_copy = seq_padded.copy()
        label = seq_copy[idx]
        seq_copy[idx] = 0
        X.append(seq_copy)
        y.append(label)


X = np.array(X)
y = np.array(y)


embeddings_index = {}

f = open('glove.6B.100d.txt')

for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs

f.close()

embedding_dim = 100
embedding_matrix = np.zeros((vocab_size, embedding_dim))

for word, i in word_index.items():
    if i < vocab_size:
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector

with strategy.scope():
    embedding_layer = Embedding(
        input_dim=vocab_size,
        output_dim=embedding_dim,
        weights=[embedding_matrix],
        trainable=True
    )

    embedding_layer.build((1,))
    embedding_layer.set_weights([embedding_matrix])

    model = Sequential([
        embedding_layer,
        Bidirectional(GRU(256,activation = 'tanh', return_sequences=True)),
        LayerNormalization(),
        Dropout(0.2),

        Bidirectional(GRU(128,activation = 'tanh', return_sequences=True)),
        LayerNormalization(),
        Dropout(0.2),

        GRU(128),
        Dropout(0.2),

        Dense(128),
        LeakyReLU(alpha=0.01),
        Dropout(0.2),

        Dense(vocab_size, activation='softmax')
    ])

    model.compile(loss='sparse_categorical_crossentropy',
                  optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                  metrics=['accuracy'])
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
model.fit(
    X,
    y,
    epochs=10,
    batch_size=64,
    verbose=1,
    validation_split=0.1
)
model.save('gru_model.h5')

test_df = pd.read_csv('Test Datas.csv')

test_sequences = tokenizer.texts_to_sequences(test_df['MASKED SENTENCES'])
test_padded = pad_sequences(test_sequences, maxlen=max_len, padding='post')

predictions = model.predict(test_padded)

predicted_words = []
for prediction in predictions:
    predicted_index = np.argmax(prediction)
    predicted_word = [word for word, idx in word_index.items() if idx == predicted_index]
    if predicted_word:
        predicted_words.append(predicted_word[0])
    else:
        predicted_words.append('Unknown')

submission_df = pd.DataFrame({
    'IDS': test_df['IDS'],
    'PREDICTED WORDS': predicted_words
})

submission_df.to_csv('submission.csv', index=False)


Running on CPU/GPU


/usr/local/lib/python3.11/dist-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


Epoch 1/10
3373/6868 ━━━━━━━━━━━━━━━━━━━━ 6:11:49 6s/step - accuracy: 0.0631 - loss: 7.5756

In [4]:
!wget https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
!unzip -q glove.6B.zip

--2025-03-16 19:23:18--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6B.zip        100%[===================>] 822.24M  5.04MB/s    in 2m 41s  

2025-03-16 19:25:58 (5.12 MB/s) - ‘glove.6B.zip’ saved [862182613/862182613]



In [ ]:
from google.colab import files

uploaded = files.upload()

KeyboardInterrupt: 

In [ ]:
!pip  install tensorflow


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 644.9/644.9 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.5/24.5 MB 71.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 110.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 100.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.3/106.3 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 107.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 5.2 MB/s eta 0:00:00
